In [10]:
import librtd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import tqdm

from Bio import SeqIO

# Read txt files in and calculate rtd metric

In [180]:
def rtd_metric(folder_path, kmer, reverse_complement, pairwise):

    files = glob.glob(folder_path + '/*.txt', 
                       recursive = True)

    if reverse_complement is True:
        constant = 4
        
    else:
        constant = 1
        
    df = pd.DataFrame(columns=list(range(constant * 2 * 4 ** kmer)))

    for seq_file in files:
        fasta_seq = SeqIO.parse(open(seq_file),'fasta')

        for fasta in fasta_seq:
            name, seq = fasta.id, str(fasta.seq)

            df.loc[len(df.index)+1] = librtd.return_time_distribution(seq, kmer, reverse_complement, pairwise).values()

    df.columns = librtd.return_time_distribution(seq, kmer, reverse_complement, pairwise).keys()
    
    return df

In [182]:
Basidiomycota = rtd_metric(folder_path = "/Users/davidchen/Documents/GitHub/rtd/Fungi/Basidiomycota", 
           kmer = 1, 
           reverse_complement = True, 
           pairwise = False)

In [184]:
Pezizomycotina = rtd_metric(folder_path = "/Users/davidchen/Documents/GitHub/rtd/Fungi/Pezizomycotina", 
           kmer = 1, 
           reverse_complement = True, 
           pairwise = False)

In [185]:
Saccharomycotina = rtd_metric(folder_path = "/Users/davidchen/Documents/GitHub/rtd/Fungi/Saccharomycotina", 
           kmer = 1, 
           reverse_complement = True, 
           pairwise = False)

In [186]:
metrics = pd.concat([Basidiomycota, Pezizomycotina, Saccharomycotina]).reset_index(drop=True)

In [187]:
metrics['Class'] = list(np.repeat(1, len(Basidiomycota))) + list(np.repeat(2, len(Pezizomycotina))) + list(np.repeat(3, len(Saccharomycotina)))

In [188]:
metrics

,C_T_mean,A_G_mean,G_A_mean,G_T_mean,T_C_std,T_A_std,G_C_mean,T_T_std,T_C_mean,T_G_mean,...,C_G_std,C_T_std,A_G_std,T_A_mean,A_T_std,T_T_mean,G_T_std,C_C_std,T_G_std,Class
0,2.667014,7.395956,3.018784,3.092291,8.154206,2.706319,7.711695,2.419480,7.839727,8.833411,...,8.564662,2.411365,7.787812,2.980677,2.888087,2.772793,2.713571,7.282014,8.648842,1
1,2.672944,8.295883,2.880475,2.760710,9.357965,2.101972,8.705148,2.131668,9.280834,8.667279,...,8.127861,2.257580,8.760953,2.592635,2.132844,2.692029,8.533418,2.229776,9.003206,1
2,3.647819,6.372130,4.085116,4.354663,6.953395,6.627751,5.901769,3.457354,6.894302,7.525039,...,6.614456,4.518717,5.959553,5.102664,6.339892,2.917200,6.202158,5.165603,7.229827,1
3,2.815146,10.841188,3.282219,3.216762,11.750963,3.105060,8.702794,2.441394,10.009733,10.906664,...,11.896670,2.793968,14.290079,3.082340,2.773327,2.710484,9.303846,2.919061,13.428412,1
4,2.768844,6.706757,2.864356,2.929001,7.130034,2.384130,6.930061,2.347134,7.069995,7.322306,...,7.324025,2.455407,7.112501,2.855972,2.473037,2.891247,2.465178,6.545671,7.431304,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,2.376514,10.109327,3.010039,2.559262,11.161648,2.201527,8.678652,1.921325,9.765070,10.733810,...,10.740350,1.908549,12.073438,2.081693,2.482343,2.576550,2.045559,9.221897,12.561378,3
220,3.280577,25.283953,3.251719,3.087318,34.285523,2.055561,15.191108,1.910273,27.243571,25.943800,...,25.811274,3.633235,30.398994,2.119923,1.958907,2.449243,22.069779,3.016197,30.672705,3
221,5.315314,31.660988,3.925637,4.976961,40.257885,2.962112,14.862153,2.335425,32.565024,32.999370,...,30.620181,7.371293,41.738863,2.296073,2.254822,2.455472,23.885711,6.662167,41.858695,3
222,2.812267,7.580595,3.017305,2.691789,10.055900,2.063361,9.081232,2.142465,8.921286,8.532322,...,9.594745,2.348371,8.962670,2.193229,2.324771,2.932453,2.222919,8.088200,10.126399,3


# Machine Learning Proof of Concept

In [88]:
from sklearn.model_selection import cross_validate
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import tree

In [92]:
def benchmark(metrics):

    scoring = ['accuracy']

    svc = svm.SVC(kernel='linear', C=1, random_state=0)
    svm_scores = cross_validate(svc, X=metrics.iloc[:, :-1], y=metrics.iloc[:, -1], scoring=scoring, cv=10)
    print("SVC: " + str(svm_scores['test_accuracy'].mean()))

    knn = KNeighborsClassifier(n_neighbors=5)
    knn_scores = cross_validate(knn, X=metrics.iloc[:, :-1], y=metrics.iloc[:, -1], scoring=scoring, cv=10)
    print("KNN: " + str(knn_scores['test_accuracy'].mean()))

    sgd = SGDClassifier(loss="hinge", penalty="l2", max_iter=100)
    sgd_scores = cross_validate(sgd, X=metrics.iloc[:, :-1], y=metrics.iloc[:, -1], scoring=scoring, cv=10)
    print("SGD: " + str(sgd_scores['test_accuracy'].mean()))

    dtree = tree.DecisionTreeClassifier()
    dtree_scores = cross_validate(dtree, X=metrics.iloc[:, :-1], y=metrics.iloc[:, -1], scoring=scoring, cv=10)
    print("Decision Tree: " + str(sgd_scores['test_accuracy'].mean()))


# K=1, RC=False, Pairwise=False

In [130]:
benchmark(metrics)

SVC: 0.8436758893280633
KNN: 0.8128458498023715
SGD: 0.7286561264822135
Decision Tree: 0.7286561264822135


# K=2, RC=False, Pairwise=False

In [123]:
benchmark(metrics)

SVC: 0.884387351778656
KNN: 0.7772727272727272
SGD: 0.716600790513834
Decision Tree: 0.716600790513834


# K=3, RC=False, Pairwise=False

In [137]:
benchmark(metrics)

SVC: 0.9021739130434783
KNN: 0.7810276679841898
SGD: 0.7824110671936759
Decision Tree: 0.7824110671936759


# K=2, RC=False, Pairwise=True

In [166]:
benchmark(metrics)

SVC: 0.807905138339921
KNN: 0.7770750988142292
SGD: 0.7723320158102768
Decision Tree: 0.7723320158102768


# K=1, RC=True, Pairwise=False

In [189]:
benchmark(metrics)

SVC: 0.8395256916996047
KNN: 0.7907114624505929
SGD: 0.7549407114624506
Decision Tree: 0.7549407114624506
